In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, Input
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
# load the word index
word_index = imdb.get_word_index()
reverse_word_index = {value: key for (key, value) in word_index.items()}

In [4]:
# load the model
model = load_model('movie_review_rnn.h5')
model.summary()

I0000 00:00:1755186469.749841   15157 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 4130 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_3 (Embedding)         │ (None, 200, 128)       │     1,280,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_3 (SimpleRNN)        │ (None, 128)            │        32,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,313,027 (5.01 MB)

 Trainable params: 1,313,025 (5.01 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [6]:
model.get_weights()

[array([[ 0.8594249 , -1.1362894 ,  0.32185364, ...,  0.18565153,
         -1.0180556 ,  0.23133054],
        [ 0.00426091, -0.09027755, -0.07101918, ..., -0.02946495,
         -0.09374371,  0.02039738],
        [ 0.03538838, -0.00863678, -0.0482873 , ..., -0.08056757,
         -0.06635261,  0.02657271],
        ...,
        [-0.01248252,  0.03080965,  0.02479442, ...,  0.01879758,
         -0.03018146,  0.05288426],
        [-0.00743155, -0.08088707,  0.01399047, ...,  0.00777753,
         -0.03591529, -0.02360426],
        [ 0.00756429,  0.12324505, -0.12510961, ..., -0.10854445,
          0.06542061, -0.11452803]], dtype=float32),
 array([[ 0.04273584, -0.14987533,  0.02558673, ...,  0.15846935,
         -0.05962821, -0.06210829],
        [ 0.0966327 ,  0.0634374 ,  0.13998857, ..., -0.00943441,
         -0.10166015,  0.0582308 ],
        [-0.01666402,  0.12396719,  0.07381713, ..., -0.10640197,
          0.1361034 ,  0.13081545],
        ...,
        [-0.13217676,  0.11628962, -0.0

In [7]:
def decode_review(text):
    return ' '.join(
        reverse_word_index.get(i - 3, '?') for i in text
    )

def preprocess_review(text):
    text = text.lower().split()
    encoded_review = [word_index.get(word, 2)+3 for word in text]
    padded_review = pad_sequences([encoded_review], maxlen=500)
    return padded_review

In [8]:
# prediction function
def predict_sentiment(review):
    preprocessd_input = preprocess_review(review)
    model.predict(preprocessd_input)
    prediction = model.predict(preprocessd_input)[0][0]
    sentiment = 'positive' if prediction > 0.5 else 'negative'
    return sentiment, prediction

In [15]:
sample_review = "This movie was fantastic! I loved it."
sentiment, score = predict_sentiment(sample_review)
print(f"Review: {sample_review}")
print(f"Sentiment: {sentiment}")
print(f"Score: {score:.4f}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
Review: This movie was fantastic! I loved it.
Sentiment: positive
Score: 0.9127
